In [1]:
# import the relevant libraries

import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg

This notebook contains the code for the stability analysis in Task A. The functions rk4_stabilty() and Euler_stabilty are from Prof. Pier Luigi Vidale and are available on blackboard. The eigenvalue stabilty analysis for the explicit scheme is of my own design.

In [2]:
# Define the global variables
# These do not change throughout the tasks

b_0 = 2.5
gamma = 0.75
c = 1
r = 0.25
alpha = 0.125
xi_2 = 0
mu_c = 2/3 # critical value of coupling parameter

# set up the non-dimensionalised constants
# set up the non-dimensionalised constants
Tnd = 7.5 # (K) SST anomalies
hnd = 150 # (m) thermocline depth
tnd = 2   # (months) time-scale

The two functions below are used to perform eigenvalue analysis for the explicit euler scheme.

In [7]:
def eigenvalue_analysis_euler(T_init, h_init, mu, e_n, xi_1, dt, nt):
    """
    Function to calculate the magnitude of the eigenvalues for the simple euler scheme.
    --------------------------------
    Input:
    T_init: initial temperature.
    h_init: initial thermocline depth.
    mu: parameter for the ROM.
    e_n: parameter for the ROM.
    xi_1: parameter for the ROM.
    dt: timestep.
    nt: number of timesteps.
    --------------------------------
    Returns:
    mag_eigenvalues: magnitude of the eigenvalues.
    """
    T_e, h_w, time = euler(T_init, h_init, mu, e_n, xi_1, dt, nt)

    # Initialize an array to store the magnitude of the eigenvalues
    mag_eigenvalues = np.zeros(nt)

    # define R
    b = b_0*mu
    R = gamma*b - c

    # Loop over each time step
    for i in range(1, nt):
        system_matrix = np.array([[1 + dt*R, dt*gamma], [dt*mu*h_w[i-1], 1 + dt*mu*R]])
        eigenvalues = np.linalg.eigvals(system_matrix)
        mag_eigenvalues[i] = np.abs(eigenvalues).max()

    return mag_eigenvalues


# write a function that will plot the magnitude of the eigenvalues for an array of timesteps (four different values of dt)
def plot_eigenvalues_euler(T_init, h_init, mu, e_n, xi_1, dt, nt):
    """
    Function to plot the magnitude of the eigenvalues for the simple euler scheme.
    --------------------------------
    Input:
    T_init: initial temperature.
    h_init: initial thermocline depth.
    mu: parameter for the ROM.
    e_n: parameter for the ROM.
    xi_1: parameter for the ROM.
    dt: timestep.
    nt: number of timesteps.
    --------------------------------
    Returns:
    None.
    """
    # create an array of values of dt
    dt_array = np.array([0.01, 0.1, 0.2, 1.0])

    # create an array to store the magnitude of the eigenvalues
    mag_eigenvalues = np.zeros((nt, len(dt_array)))

    # loop over each value of dt
    for i in range(len(dt_array)):
        # calculate the magnitude of the eigenvalues
        mag_eigenvalues[:, i] = eigenvalue_analysis_euler(T_init, h_init, mu, e_n, xi_1, dt_array[i], nt)

    # create an array of time
    time = np.linspace(0,41,nt)

    # create a figure
    fig = plt.figure()
    # create an axis
    ax = fig.add_subplot(111)
    # plot the magnitude of the eigenvalues for each value of dt
    ax.plot(time, mag_eigenvalues[:, 0], label='dt = 0.001')
    ax.plot(time, mag_eigenvalues[:, 1], label='dt = 0.01')
    ax.plot(time, mag_eigenvalues[:, 2], label='dt = 0.1')
    ax.plot(time, mag_eigenvalues[:, 3], label='dt = 1.0')
    # add a legend
    ax.legend(loc='upper right')
    # label the x-axis
    ax.set_xlabel('Time (months)')
    # label the y-axis
    ax.set_ylabel('|A|')
    # add a title
    #ax.set_title('Magnitude of the eigenvalues for the simple euler scheme')
    # set y-axis limits
    ax.set_ylim(1.0, 1.3)
    # show the plot
    plt.show()
    # save the figure
    fig.savefig('ROM_linear_neutral_final_test_eigenvalues_euler.png')

The functions below were written by Prof. Pier Luigi Vidale.

In [4]:
# Pierre-Luigi-Vidale's code for stability analysis of the RK4 scheme
def rk4_stability():
    """
    Stability of rk4 for the linear recharge oscillator. Author: Prof. Pier Luigi Vidale.
    """

    x = np.arange(-4,2,0.01)
    y = np.arange(-3,3,0.01)
    [X,Y] = np.meshgrid(x,y)
    z = X + 1j*Y

    A = 1 + z + 0.5*z**2 + 1/6*z**3 + 1/24*z**4
    Aabs = abs(A)

    plt.contourf(X, Y, Aabs, np.linspace(0,1,11))
    #    plt.xlabel('Re[$\lambda$]'); plt.ylabel('Im[$\lambda$]')
    plt.xlabel('$\Re(\lambda) \Delta t$'); plt.ylabel('$\Im(\lambda) \Delta t$')
    cbar = plt.colorbar()
    cbar.set_label("|A|", rotation=0)

# PLV's code for stability analysis of the Euler scheme (+ modification by BWH)
def Euler_stability():
    "Stability of Explict Euler for the linear recharge oscillator. Author: Prof. Pier Luigi Vidale"

    x = np.arange(-4,2,0.01)
    y = np.arange(-3,3,0.01)
    [X,Y] = np.meshgrid(x,y)
    z = X + 1j*Y

    # CPL for the linear recharge oscillator euler scheme
    A = 1 + z
    Aabs = abs(A)

    plt.contourf(X, Y, Aabs, np.linspace(0,1,11), cmap='jet')
    #    pl.xlabel('$\lambda \Delta t$'); plt.ylabel('$\omega \Delta t$ ')
    plt.xlabel('$\Re(\lambda) \Delta t$'); plt.ylabel('$\Im(\lambda) \Delta t$')
    # use a different colorbar
    cbar = plt.colorbar()
    #cbar = plt.colorbar()
    cbar.set_label("|A|", rotation=0)


In [5]:
# define a function for the stability analysis in Task A
def Task_A_stability_analysis(dt=0.02, no_periods=1, len_period=41):
    """
    Function to perform stability analysis for the simple euler scheme.
    --------------------------------
    Input:
    None.
    --------------------------------
    Returns:
    None.
    """
    # define the initial conditions
    T_init = 1.125/Tnd
    h_init = 0.0/hnd
    # define the parameters
    mu = mu_c = 2/3
    e_n = 0.0
    xi_1 = 0.0
    # define the timestep
    dt = dt
    # define the number of timesteps
    nt = int(no_periods*len_period/dt)
    # call the function to plot the magnitude of the eigenvalues
    plot_eigenvalues_euler(T_init, h_init, mu, e_n, xi_1, dt, nt)

    # call the function to plot the stabiity of the RK4 scheme
    rk4_stability()

    # call the function to plot the stabiity of the Euler scheme
    Euler_stability()